<a href="https://colab.research.google.com/github/kavyakapoor200/custom-chatbot/blob/main/custom_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install langchain flask openai chromadb beautifulsoup4 requests
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [7]:
from langchain.document_loaders import WebBaseLoader

# Load data from the website
loader = WebBaseLoader("https://brainlox.com/courses/category/technical")
docs = loader.load()

# Print sample extracted data
print(docs[:3])


[Document(metadata={'source': 'https://brainlox.com/courses/category/technical', 'title': 'Brainlox: Learn technical courses.', 'description': 'Your one stop education destination!', 'language': 'zxx'}, page_content='Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you$30per sessionLEARN SCRATCH PROGRAMING\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details$30per sessionLEARN CLOUD COMPUTING BASICS-AWS\nIn this course we are going to cover the basics and the most important services on AWS,\nAt the end  20 LessonsView Details$30per sessionLEARN MOBILE DEVELOPMENT\nMobile application development is the process of creating software applications that run on a mobil 24 LessonsView Details$30per sessionLEARN CORE JAVA PROGRAMMING ONLINE\nJava is a very popular high-level, class-based, object-orie

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Use a free Hugging Face embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert documents into embeddings and store in ChromaDB
vectorstore = Chroma.from_documents(docs, embeddings)

# Save the vectorstore
vectorstore.persist()


<ipython-input-9-aa525f89cbe8>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-aa525f89cbe8>:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from flask import Flask, request, jsonify
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Hugging Face API key (if needed)
HF_TOKEN = "your_huggingface_api_key"

app = Flask(__name__)

# Load stored vector database
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = vectorstore.as_retriever()

# Use Google's Free LLM: Gemma-7B
model_name = "google/gemma-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_name, token=HF_TOKEN, torch_dtype="auto")

# Create a text-generation pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Conversation Memory for better responses
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Use ConversationalRetrievalChain instead of RetrievalQA
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

@app.route("/chat", methods=["POST"])
def chat():
    user_query = request.json["query"]
    response = qa_chain.run({"question": user_query, "chat_history": []})
    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(debug=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


<ipython-input-15-96926c33ef74>:29: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
